# Make Database Connection

In [1]:
from commandcast.data_manager import DataManager, DataManagerConfig

db = DataManager(
    host='localhost',
    port=9000
)

# Ingest Data

In [2]:
import pandas as pd

df = pd.read_csv("data/m4.csv")
df.head()

df['ds'] = pd.to_datetime(df['ds'])
df.dtypes

# ID Columns 
ds_col = 'ds'
id_col = 'unique_id'

# Measurement Columns
measure_cols = list(df.drop(['ds', 'unique_id'], axis=1).columns)        

# Configure Dataset in Database
config = DataManagerConfig(**{
    'dataset_name' : 'M4_HOURLY', # Overall name of the timeseries dataset
    'feature_engineering': 'minimal',
    'ds_col' : ds_col, # Timeseries column
    'id_col' : id_col, # Identifier Unique to Timeseries,
    'hierarchy': [],
    'measure_cols': measure_cols
})

# Method to Create Tables and Ingest Data
db.create_dataset(config=config, df=df)

2024-12-18 11:32:25.453 | WARNING  | commandcast.data_manager:create_dataset:163 - Timeseries and features table already exists. Aborting ingest.


# Explore how to create time series featues

In [3]:
features = db.create_features(df, config=config)
features.head()

Feature Extraction: 100%|██████████| 25/25 [00:00<00:00, 33.63it/s]


,unique_id,time_begin,time_end,count,dataset_name,values__sum_values,values__median,values__mean,values__length,values__standard_deviation,values__variance,values__root_mean_square,values__maximum,values__absolute_maximum,values__minimum
0,T1,2015-07-01 12:00:00,2015-08-01 15:00:00,748,M4_HOURLY,478586.0,634.0,639.820856,748.0,156.533479,24502.729939,658.690714,926.0,926.0,349.0
1,T10,2015-07-01 12:00:00,2015-08-01 15:00:00,748,M4_HOURLY,331441.0,442.0,443.102941,748.0,35.451199,1256.787531,444.518846,517.0,517.0,336.0
2,T100,2015-07-01 12:00:00,2015-08-01 15:00:00,748,M4_HOURLY,753242.5,964.0,1007.008690,748.0,386.660351,149506.226863,1078.690284,1875.0,1875.0,351.0
3,T101,2015-07-01 12:00:00,2015-08-01 15:00:00,748,M4_HOURLY,1777609.0,2374.0,2376.482620,748.0,247.417708,61215.522425,2389.327346,2992.0,2992.0,1533.0
4,T102,2015-07-01 12:00:00,2015-08-01 15:00:00,748,M4_HOURLY,1305922.0,1739.0,1745.885027,748.0,425.193520,180789.529562,1796.915150,2540.0,2540.0,634.0


# Extract a time series from the database

In [4]:
id = 'T2'
features = db.get_features(id, table_name = config['ft_table_name'])
print(features)
series = db.get_series(id, table_name = config['ts_table_name'])
print(series)

  unique_id dataset_name                     time_end  count  \
0        T2    M4_HOURLY  2015-08-01T15:00:00.000000Z    748   

   values__sum_values  values__median  values__mean  values__length  \
0           1943707.0          2517.0    2598.53877           748.0   

   values__standard_deviation  values__variance  values__root_mean_square  \
0                  655.316397     429439.580048                2679.89614   

   values__maximum  values__absolute_maximum  values__minimum  \
0           4013.0                    4013.0           1466.0   

                     timestamp  
0  2015-07-01T12:00:00.000000Z  
    unique_id  values                           ds
0          T2  3124.0  2015-07-01T12:00:00.000000Z
1          T2  2990.0  2015-07-01T13:00:00.000000Z
2          T2  2862.0  2015-07-01T14:00:00.000000Z
3          T2  2809.0  2015-07-01T15:00:00.000000Z
4          T2  2544.0  2015-07-01T16:00:00.000000Z
..        ...     ...                          ...
743        T2  3558

In [5]:
import plotly.express as px

# Create a Plotly figure
fig = px.line(series, x='ds', y='values', title=id, labels={'ds': 'Date', 'values': 'Values'})

# Update layout for better readability
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Values',
    xaxis=dict(showgrid=True, tickangle=45),
    yaxis=dict(showgrid=True),
    template='plotly_dark'
)

# Show the figure
fig.show()

# Temporal Agents

- StatisticalForecast
- NeuralForecast
- FoundationalForecast
- ForecastCombination
- ForecastReconciliation
- CodeExecutor



In [6]:
import numpy as np

class GaussianReconciliation:
    def __init__(self, Sct, covariance_matrix):
        """
        Initialize the Gaussian reconciliation with structural matrix Sct and covariance matrix.

        Parameters:
        - Sct: Cross-temporal structural matrix.
        - covariance_matrix: Covariance matrix of the base forecasts.
        """
        self.Sct = Sct
        self.covariance_matrix = covariance_matrix
        self.inv_covariance = np.linalg.inv(covariance_matrix)

    def reconcile_forecasts(self, base_forecasts):
        """
        Reconcile base forecasts using the projection matrix method.

        Parameters:
        - base_forecasts: Array of base forecasts (incoherent forecasts).

        Returns:
        - reconciled_forecasts: Reconciled forecasts.
        """
        # Calculate the projection matrix M
        Sct_cov_Sct_T_inv = np.linalg.inv(self.Sct @ self.inv_covariance @ self.Sct.T)
        M = self.Sct.T @ Sct_cov_Sct_T_inv @ self.Sct @ self.inv_covariance
        
        # Reconcile the forecasts
        reconciled_forecasts = M @ base_forecasts
        return reconciled_forecasts

    def reconcile_covariance(self):
        """
        Compute the reconciled covariance matrix.

        Returns:
        - reconciled_covariance: Reconciled covariance matrix.
        """
        # Calculate the projection matrix M
        Sct_cov_Sct_T_inv = np.linalg.inv(self.Sct @ self.inv_covariance @ self.Sct.T)
        M = self.Sct.T @ Sct_cov_Sct_T_inv @ self.Sct @ self.inv_covariance
        
        # Reconcile the covariance matrix
        reconciled_covariance = M @ self.covariance_matrix @ M.T
        return reconciled_covariance

# Example Usage
# Assume Sct is the structural matrix, covariance_matrix is the covariance matrix of base forecasts, 
# and base_forecasts is the vector of base forecasts.

# Update the structural matrix to match the dimensionality of the problem
Sct = np.array([[1, 0, 1], [0, 1, 1], [1, 1, 0]])  # Updated structural matrix
covariance_matrix = np.array([[1.0, 0.5, 0.3], [0.5, 1.0, 0.2], [0.3, 0.2, 1.0]])  # Covariance matrix
base_forecasts = np.array([100, 200, 300])  # Base forecasts

# Create the reconciliation object
reconciliation = GaussianReconciliation(Sct, covariance_matrix)

# Get the reconciled forecasts
reconciled_forecasts = reconciliation.reconcile_forecasts(base_forecasts)
reconciled_covariance = reconciliation.reconcile_covariance()

print("Reconciled Forecasts:", reconciled_forecasts)
print("Reconciled Covariance Matrix:\n", reconciled_covariance)


Reconciled Forecasts: [100. 200. 300.]
Reconciled Covariance Matrix:
 [[1.  0.5 0.3]
 [0.5 1.  0.2]
 [0.3 0.2 1. ]]


In [7]:
Sct

array([[1, 0, 1],
       [0, 1, 1],
       [1, 1, 0]])

In [8]:
base_forecasts

array([100, 200, 300])

In [9]:
covariance_matrix

array([[1. , 0.5, 0.3],
       [0.5, 1. , 0.2],
       [0.3, 0.2, 1. ]])